In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_3.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))


In [3]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [4]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [5]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [7]:
# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [8]:
# Build LSTM model
lstm_model = Sequential()

# Input layer
lstm_model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))

# Add LSTM layer and freeze it (trainable=False)
lstm_layer = LSTM(50, trainable=False)  # Freezing LSTM layer
lstm_model.add(lstm_layer)

# Dropout layer (kept trainable)
lstm_model.add(Dropout(0.2))

# Output Dense layer (trainable)
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [9]:
# Compile model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
# Train model
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0892 - loss: 1.8526 - val_accuracy: 0.0423 - val_loss: 1.7434
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1274 - loss: 1.7462 - val_accuracy: 0.0529 - val_loss: 1.6501
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1463 - loss: 1.6351 - val_accuracy: 0.1799 - val_loss: 1.5726
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2093 - loss: 1.5675 - val_accuracy: 0.3333 - val_loss: 1.5058
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3222 - loss: 1.4736 - val_accuracy: 0.3598 - val_loss: 1.4513
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4149 - loss: 1.4177 - val_accuracy: 0.4180 - val_loss: 1.4035
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4443 - loss: 1.3748 - val_accuracy: 0.4339 - val_loss: 1.3630
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4630 - loss: 1.3410 - val_accuracy: 0.4339 - val_loss

In [11]:
# Save the training history
with open('training_history_freeze.json', 'w') as f:
    json.dump(lstm_history.history, f)


In [12]:
# Save LSTM model
lstm_model.save('verification_model_freeze.h5')